# Lemma Tutorial

This tutorial will take you on a tour of what you can achieve with Lemma, including:

* Using the Lemma syntax to write mathematical expressions that can be
  both formatted as LaTeX and evaluated in Python 
* Defining formulas that can be used and tested from Python code.
* Check that each step in your algebraic transformation results in an equivalent expression.
* Extending the Lemma syntax with your own custom operators.



## Lemma is built on Hy

To get the most out of this tutorial, you should have a basic familiarity with the Hy programming language, which Lemma uses as the foundation for its syntax. Hy is essentially a Lisp syntax on top of the Python programming language, so if you're familiar with Python you should find learning Hy quite easy.

If you're not familiar with Hy, you should have a quick read through the [Hy Tutorial](https://docs.hylang.org/en/stable/tutorial.html) or [Learn Hy in Y Minutes](https://learnxinyminutes.com/docs/hy/).

## Installation and Importing

Install Lemma with `pip`:

In [1]:
%%bash

python -m pip install lemma

When working from a Jupyter notebook, load the `lemma.ipython` extension to use `%%hy` and `%%lemma` [magics](https://ben-denham.github.io/lemma/#/installation?id=jupyteripython-notebooks) and enable automatic formatting of Lemma objects.

In [2]:
%load_ext lemma.ipython

To use Lemma in a `.hy` file or notebook, require `lemma.core` and load the operators you want to use from their respective "domain" modules:

In [3]:
%%hy
;; Use the %%hy cell magic to evaluate Hy code

(require [lemma.core :as le])
(import [lemma.domain.algebra [seq-sum pow length add sub PI
                               div/frac :as div pow
                               mul/times :as mul]])

## Lemma Expressions

We can express mathematical operations as Lemma *expressions*.

An expression can be rendered as **LaTeX math notation** AND **evaluated to a Python/Hy value**:

In [4]:
%%lemma
;; Use the %%lemma cell magic to directly evaluate Lemma expressions

PI

(add 3 0.14)

(div 22 7)

(seq-sum
    [x [6 0.2 0.08]]
    (div x 2))

$$\pi$$,3.141592653589793
$$3 + 0.14$$,3.14
$$\frac{22}{7}$$,3.142857142857143
"$$\sum_{x \in \{6, 0.2, 0.08\}} \frac{x}{2}$$",3.14


Use `le.expr` to define an expression in Hy code:

In [5]:
%%hy

;; `setv` assigns the expression to `my-expr`
(setv my-expr (le.expr (div 1 2)))

my-expr

$$\frac{1}{2}$$,0.5


Get an expression's LaTeX with the `.latex` method, and get its value by calling it as a function:

In [6]:
%%hy

(print "LaTeX notation:" (.latex my-expr))
(print "Value:" (my-expr))

LaTeX notation: \frac{1}{2}
Value: 0.5


## Python Interop

Because Lemma objects are just Hy/Python objects, they can be used directly from Python code:

In [7]:
# Note that hyphens (-) in Hy variable names are replaced by underscores in Python:
print("LaTeX notation:", my_expr.latex())
print("Value:", my_expr())

# Use the %lemma inline cell magic to embed expressions in Python code:
new_expr = %lemma (add PI 1)
new_expr

LaTeX notation: \frac{1}{2}
Value: 0.5


$$\pi + 1$$,4.141592653589793


## Order of operations

Hy automatically inserts parentheses in LaTeX notation to accurately represent the order of operations:

In [8]:
%%lemma

(sub (add 1 2) 3)

(sub 1 (add 2 3))

$$\left(1 + 2\right) - 3$$,0
$$1 - \left(2 + 3\right)$$,-4


Use `#p` to force the insertion of parentheses, and `#b` to explicitly remove them:

In [9]:
%%lemma

(add (add 1 2) (div 2 3))

(add (add 1 2) #p(div 2 3))

(add #b(add 1 2) (div 2 3))

$$\left(1 + 2\right) + \frac{2}{3}$$,3.6666666666666665
$$\left(1 + 2\right) + \left(\frac{2}{3}\right)$$,3.6666666666666665
$$1 + 2 + \frac{2}{3}$$,3.6666666666666665


## Lemma Identifiers



When Lemma needs to name an unbound variable in math notation, it defaults to using the name of the associated Hy symbol.

For example, `b` in `seq-sum` below:

In [10]:
%%lemma

(seq-sum [b [1 2 3]] b)

"$$\sum_{b \in \{1, 2, 3\}} b$$",6


A Lemma *identifier* can be declared to specify the LaTeX notation to use instead, wherever a symbol is used:

In [11]:
%%hy

(le.def-identifier beta r"\beta")

In [12]:
%%lemma

(seq-sum [beta [1 2 3]] beta)

"$$\sum_{\beta \in \{1, 2, 3\}} \beta$$",6


## Lemma Formulas

Lemma *formulas* enable you to use the Lemma expression syntax to define reusable functions that can be executed and tested from Hy/Python code, and also used to generate LaTeX notation for documentation.

A formula is similar to a normal Hy/Python function, but with a Lemma expression for its body:

In [13]:
%%hy

(le.def-identifier xs "X")
(le.def-identifier mu r"\mu")

(le.def-formula variance r"\sigma^2"
  [xs mu n]
  "Formula for population variance."
  (div
   (seq-sum [x xs]
    (pow (sub x mu) 2))
   n))

Formulas can be used like any other operator in a Lemma expression:

In [14]:
%%lemma

(variance [1 2 3] :mu 2 :n 3)

;; Using formula.op formats the formula using the notation of its definition body:
(variance.op [1 2 3] :mu 2 :n 3)

"$$\sigma^2\left(\{1, 2, 3\}, 2, 3\right)$$",0.6666666666666666
"$$\frac{\sum_{x \in \{1, 2, 3\}} \left(x - 2\right)^{2}}{3}$$",0.6666666666666666


Formulas can also be called directly like a Hy/Python function:

In [15]:
variance(xs=[0, 5, 10], mu=5, n=3)

16.666666666666668

Calling the `.latex` method of a formula without any arguments returns its full definition:

In [16]:
variance.latex()

'\\sigma^2\\left(X, \\mu, n\\right) = \\frac{\\sum_{x \\in X} \\left(x - \\mu\\right)^{2}}{n}'

Formulas have full support for positional, keyword, and optional arguments, see [the docs](https://ben-denham.github.io/lemma/#/lemma.core?id=formulas) for more details.

## Lemma Equations

Lemma *equations* allow you to define a single parameterised computation with multiple equivalent Lemma expressions. Because Lemma will raise an exception if any of the expressions evaluate to a different result, they are a great way to check that all of the steps in an algebraic transformation are equivalent:

In [17]:
%%hy

(le.def-equation my-equation
    [x]
    "Equation involving FOIL expansion."
    [#b(mul (add x 1) (sub x 1))
     (sub #b(add #b(sub (pow x 2) x) x) 1)
     (sub (pow x 2) 1)])

Equations are similar to formulas in that they can be used as Hy/Python functions or from Lemma expressions:

In [18]:
%%hy

(display (.latex my-equation))
(display (.latex my-equation :x 5))
(print (my-equation :x 5))

'\\begin{aligned} \\left(x + 1\\right) \\times \\left(x - 1\\right) &= x^{2} - x + x - 1 \\\\&= x^{2} - 1\\end{aligned}'

'\\begin{aligned} \\left(5 + 1\\right) \\times \\left(5 - 1\\right) &= 5^{2} - 5 + 5 - 1 \\\\&= 5^{2} - 1\\end{aligned}'

24


In [19]:
%%lemma

;; When used as an operator, the first expression in the definition of an equation is used.
(add (my-equation :x 5) 4)

$$\left(5 + 1\right) \times \left(5 - 1\right) + 4$$,28


Lemma raises an exception if any of the expressions do not evaluate to the same result:

In [20]:
%%hy

(import [lemma.exceptions [LeEquationError]])
(le.def-equation bad-equation [x]
                 [(add x 1)
                  (sub x 1)])

(display (.latex bad-equation))

;; When executing an equation, all expressions are executed,
;; and an exception is raised if any of the results were not equal.
(try
 (bad-equation :x 5)
 (except [ex LeEquationError]
  (print ex)))

'\\begin{aligned} x + 1 &= x - 1\\end{aligned}'

While evaluating LeEquation#bad-equation with arguments [x=5]: result '4' of LeExpression#(LeCallableOperator#sub 5 1) did not equal result '6' of LeExpression#(LeCallableOperator#sub 5 1)


## Extending Lemma

Lemma is designed to be an extensible language. Allowing you to specify notation for all the computations you need in your target mathematical domain.

You can define your own constants with `le.def-constant`:

In [21]:
%%hy

(import math)
(le.def-constant e "e" math.e)

In [22]:
%%lemma

(add e 1)

$$e + 1$$,3.718281828459045


You can even define your own operators in terms of Lemma expressions...

In [23]:
%%hy

;; Define operators using lemma expressions.
(le.def-operator decrement [val]
  (expr (sub val 1)))

In [24]:
%%lemma

(decrement 3)

$$3 - 1$$,2


...Hy expressions with custom LaTeX...

In [25]:
%%hy

(le.def-operator increment [val]
  (precedence 100)
  (latex f"{val} + 1")
  (hy (+ val 1)))

In [26]:
%%lemma

(increment 3)

$$3 + 1$$,4


...or even macros:

In [27]:
%%hy

(import [lemma.lang [gen-latex gen-hy]])

;; Define operators using Hy macros (arguments are pass-by-name, and
;; hy-macro is expected to return a quoted Hy expression). Useful for
;; more complex operators that don't just take lemma expressions as
;; arguments (like seq-sum).
(le.def-operator plus2 [val]
  (precedence 100)
  (latex-macro
   f"{(gen-latex val)} + 2")
  (hy-macro
   `(+ ~(gen-hy val) 2)))

In [28]:
%%lemma

(plus2 3)

$$3 + 2$$,5


More information about extending Lemma is available in [the docs](https://ben-denham.github.io/lemma/#/lemma.core?id=defining-custom-operators).

## Next Steps

* [API reference](https://ben-denham.github.io/lemma/#/lemma.core)
* [Source-code for `lemma.domain.algebra`](https://github.com/ben-denham/lemma/blob/master/lemma/domain/algebra.hy) (useful reference for writing your own domains)
* [Documentation home](https://ben-denham.github.io/lemma)